# Домашнее задание: Линейные модели. Начало

В этом домашнем задании будем работать с датасетом `diabetes` - предсказывать уровень сахара в крови пациентов по определенным показателям.

Загрузим набор данных и сохраним информацию о признаках в переменную `X`, а о зависимой переменной – в переменную `y`.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)

X = data.data
y = data.target

X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


## Задание 1

Есть ли в наборе данных пропущенные значения? Если да, то удалите их.

Сколько строк после удаления стало в датасете?

In [12]:
# your code here
X.shape

(442, 10)

Используя функцию `train_test_split()`, разделите выборку на тренировочную и тестовую, и долю тестовой выборки задайте равной 0.3. Так как разбиение осуществляется случайным образом, не забудьте зафиксировать `random_state=42` для воспроизводимости результатов.

In [14]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=42, test_size=0.3)

## Задание 2

На тренировочной выборке обучите шесть классификаторов kNN, отличающихся только числом соседей. Для первого классификатора число соседей поставьте равным 1, для второго - 3, для третьего – 5, для четвертого – 10, для пятого – 15 и для шестого – 25 (обратите внимание на параметр `n_neighbours` класса `KNeighborsRegressor`).

Вычислите значение метрики $R^2$ на тесте для каждой из моделей. Чему равно значение метрики для лучшей модели? Ответ округлите до сотых.

In [15]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

In [16]:
# your code here
n_neighbors_list = [1, 3, 5, 10, 15, 25]

# Словарь для сохранения значений метрики R^2 для каждого классификатора
r2_scores = {}

# Обучение и оценка моделей для различных чисел соседей
for n_neighbors in n_neighbors_list:
    knn_regressor = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn_regressor.fit(Xtrain, ytrain)
    y_pred = knn_regressor.predict(Xtest)
    r2_scores[n_neighbors] = r2_score(ytest, y_pred)

# Нахождение лучшей модели по значению метрики R^2
best_n_neighbors = max(r2_scores, key=r2_scores.get)
best_r2_score = r2_scores[best_n_neighbors]

print(f"Лучшее значение метрики R^2: {best_r2_score:.2f} для {best_n_neighbors} соседей.")

Лучшее значение метрики R^2: 0.45 для 25 соседей.


## Задание 3

Теперь обучите линейную регрессию (LineaRegression) на тренировочных данных и выведите $R^2$ на тесте, ответ округлите до сотых. Какое значение метрики получилось?

In [17]:
from sklearn.linear_model import LinearRegression

# your code here
linear_regression = LinearRegression()
linear_regression.fit(Xtrain, ytrain)
y_pred = linear_regression.predict(Xtest)
r2_linear_regression = r2_score(ytest, y_pred)
r2_linear_regression

0.4772897164322617

## Задание 4

Обучите SGDRegressor с гиперпараметрами по умолчанию на тренировочных данных и выведите $R^2$ на тесте, ответ округлите до сотых. Какое значение метрики получилось?

In [18]:
from sklearn.linear_model import SGDRegressor

# your code here
sgdr = SGDRegressor()
sgdr.fit(Xtrain, ytrain)
y_pred = sgdr.predict(Xtest)
r2_sgdr = r2_score(ytest, y_pred)
r2_sgdr

/Users/magewade/Desktop/LinearRegression/.venv/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:1575: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


0.41659585106266095

## Задание 5

Подберите оптимальный градиентный шаг `eta0` для SGDRegressor. Перебирайте шаг в диапазоне от 0.00001 до 0.1 не включительно с шагом 0.0001.

Какого наилучшего качества на тесте удалось добиться? Полученное значение $R^2$ округлите до сотых.

In [30]:
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import GridSearchCV

# Отключение предупреждений
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# your code here
#  0.00001 до 0.1 не включительно с шагом 0.0001.
r2_sgdr_scores = {}
steps = [0.00001, 0.0001, 0.001, 0.01, 0.1]


sgdr = SGDRegressor(random_state=42)

param_grid = {'eta0': steps}

grid_search = GridSearchCV(estimator=sgdr, param_grid=param_grid, cv=5, verbose=2)

grid_search.fit(Xtrain, ytrain)

best_eta0 = grid_search.best_params_['eta0']

best_model = grid_search.best_estimator_

y_pred = best_model.predict(Xtest)

best_r2_score = r2_score(ytest, y_pred)

print(f"Наилучшее значение R^2: {best_r2_score:.3f} для eta0={best_eta0:.5f}.")


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END .........................................eta0=1e-05; total time=   0.0s
[CV] END .........................................eta0=1e-05; total time=   0.0s
[CV] END .........................................eta0=1e-05; total time=   0.0s
[CV] END .........................................eta0=1e-05; total time=   0.0s
[CV] END .........................................eta0=1e-05; total time=   0.0s
[CV] END ........................................eta0=0.0001; total time=   0.0s
[CV] END ........................................eta0=0.0001; total time=   0.0s
[CV] END ........................................eta0=0.0001; total time=   0.0s
[CV] END ........................................eta0=0.0001; total time=   0.0s
[CV] END ........................................eta0=0.0001; total time=   0.0s
[CV] END .........................................eta0=0.001; total time=   0.0s
[CV] END ........................................

## Задание 6

Обучите SGDRegressor с функцией потерь MAE (другие гиперпараметры оставьте по умолчанию). Чему равен $R^2$ на тесте? Ответ округлите до сотых.

P.S. Зафиксируйте `random_state=42`.

In [32]:
# your code here
sgdr = SGDRegressor(loss='epsilon_insensitive', random_state=42)
sgdr.fit(Xtrain, ytrain)
y_pred = sgdr.predict(Xtest)
r2_sgdr = r2_score(ytest, y_pred)
r2_sgdr

-0.2420156113369718

## Задание 7

Выведите на экран веса наилучшей из линейных моделей.

Какой признак имеет наибольший по модулю вес?

In [ ]:
# your code here